In [1]:
# datasets: Biblioteca da Hugging Face para carregar e manipular conjuntos de dados
# evaluate: Ferramenta para métricas de avaliação em Machine Learning
# scikit-learn: Biblioteca de aprendizado de máquina com algoritmos e métricas
# transformers: Modelos pré-treinados da Hugging Face para NLP e outras tarefas

!pip install datasets 
!ip install evaluate 
!pip install scikit-learn 
!pip install transformers 

'ip' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [2]:
# Importa a biblioteca pandas para manipulação de dados tabulares
import pandas as pd

# Importa a função train_test_split do sklearn para dividir os dados em conjuntos de treino e teste
from sklearn.model_selection import train_test_split

# Importa as classes Dataset e DatasetDict da biblioteca datasets para estruturar os dados de forma compatível com a Hugging Face
from datasets import Dataset, DatasetDict

# Importa a classe AutoTokenizer da biblioteca transformers para tokenização automática do texto
from transformers import AutoTokenizer

# Importa a classe DataCollatorWithPadding para garantir padding uniforme nos lotes de dados
from transformers import DataCollatorWithPadding

# Importa a classe AutoModelForSequenceClassification para carregar um modelo pré-treinado para classificação de sequência
# Importa a classe TrainingArguments para definir os parâmetros de treinamento do modelo
# Importa a classe Trainer para facilitar o treinamento do modelo
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# Importa as funções accuracy_score e classification_report do sklearn para avaliar o desempenho do modelo
from sklearn.metrics import accuracy_score, classification_report

# Importa a biblioteca evaluate para calcular métricas de avaliação de modelos de Machine Learning
import evaluate

# Importa a biblioteca NumPy para manipulação de arrays numéricos
import numpy as np

# Importa a classe EarlyStoppingCallback da biblioteca transformers para interromper o treinamento
# caso a métrica monitorada não melhore após um número definido de épocas
from transformers import EarlyStoppingCallback

In [3]:
# Dados
data =  pd.read_csv(r"C:\Users\willgnnerferreira\Documents\Modelos\neuralmindbert-large-portuguese-cased\Dados\dataset_FINAL.csv")

In [ ]:
# Usa apenas 50% dos dados totais para reduzir o tempo de processamento
data_sampled = data.sample(frac=0.5, random_state=42)  # Seleciona 50% aleatoriamente

# Divide o dataset em treino (80%) e teste (20%) dentro dos 40% escolhidos
train, test = train_test_split(data_sampled, 
                               test_size=0.2,  # 20% para teste, 80% para treino
                               stratify=data_sampled["label"],  # Mantém a proporção das classes
                               random_state=42)

# Exibe o tamanho dos novos conjuntos reduzidos
print(train.shape, test.shape)

(10712, 3) (2678, 3)


In [5]:
# Cria um objeto DatasetDict (estrutura de dados do Hugging Face) para armazenar os conjuntos de treino e teste
dataset = DatasetDict({
    # Converte o DataFrame de treino para um formato compatível com a biblioteca datasets
    "train": Dataset.from_pandas(train[["text", "label"]], preserve_index=False),
    
    # Converte o DataFrame de teste para um formato compatível com a biblioteca datasets
    "test": Dataset.from_pandas(test[["text", "label"]], preserve_index=False)
})

In [6]:
# neuralmind/bert-large-portuguese-cased
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-large-portuguese-cased", model_max_length=512)

In [7]:
# Função de pré-processamento para tokenizar os textos
def preprocess_function(examples):
    # Aplica o tokenizador ao texto, cortando se ultrapassar o tamanho máximo permitido
    return tokenizer(examples["text"], truncation=True)

In [8]:
# Aplica a função de pré-processamento ao dataset, processando os dados em lotes
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/10712 [00:00<?, ? examples/s]

Map:   0%|          | 0/2678 [00:00<?, ? examples/s]

In [9]:
# Cria um Data Collator para adicionar padding automaticamente aos inputs
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
# Carrega a métrica de acurácia da biblioteca evaluate
accuracy = evaluate.load("accuracy")

In [11]:
# Função para calcular métricas de avaliação do modelo
def compute_metrics(eval_pred):
    # Desempacota as previsões e os rótulos verdadeiros (labels)
    predictions, labels = eval_pred

    # Obtém o índice da classe com maior probabilidade para cada previsão
    predictions = np.argmax(predictions, axis=1)

    # Calcula a acurácia comparando as previsões com os rótulos reais
    return accuracy.compute(predictions=predictions, references=labels)

In [12]:
# Dicionário que mapeia os IDs das classes para seus respectivos rótulos
id2label = {
    1: "FATO",   # Classe 1 representa "FATO"
    0: "TESE",   # Classe 0 representa "TESE"
    2: "RUIDO"   # Classe 2 representa "RUIDO"
}

# Dicionário inverso que mapeia os rótulos das classes para seus respectivos IDs numéricos
label2id = {
    "FATO": 1,   # "FATO" é representado pelo ID 1
    "TESE": 0,   # "TESE" é representado pelo ID 0
    "RUIDO": 2   # "RUIDO" é representado pelo ID 2
}

In [13]:
# Mostrar algumas amostras do dataset com rótulos traduzidos
for i in range(5):  # Exibe 5 exemplos
    text = dataset["train"][i]["text"]
    label_id = dataset["train"][i]["label"]
    label_name = id2label.get(label_id, "Desconhecido")  # Converte ID para nome
    
    print(f"Texto: {text}\nLabel ID: {label_id} -> Classe: {label_name}\n{'-'*50}")

Texto: a requerente é estudante e com isso possui um cartão de crédito universitário do banco do brasil. na tarde do dia 15 de junho de 2022, a autora tentou realizar uma compra por meio de seu cartão de crédito, e a mesma foi negada. ocorre que o cartão estaria funcionando normalmente e havia limite disponível. bastante envergonhada com a sua honra e sua imagem no momento da compra, até porque o único meio de pagamento que esta tinha era o seu cartão de crédito, ligou imediatamente ao banco do brasil para tentar solucionar o problema. a autora foi informada pelo banco que o cartão havia sido suspenso em virtude de uma negativação que estaria constando em seu nome de uma outra empresa financeira, o que a surpreendeu. não acreditando na negativação indevida, a autora imediatamente acessou o aplicativo serasa, do qual para sua surpresa continha a restrição inserida pela empresa avista s/a credito financiamento e investimento, inscrita no cnpj sob o n° 23.862.762/0001-00 conforme print ab

In [14]:
# Carrega o modelo pré-treinado BERT para classificação de sequência
model = AutoModelForSequenceClassification.from_pretrained(
    "neuralmind/bert-large-portuguese-cased",  # Nome do modelo pré-treinado em português
    num_labels=3,  # Número de classes na classificação (FATO, TESE, RUIDO)
    id2label=id2label,  # Dicionário que mapeia IDs para nomes das classes
    label2id=label2id   # Dicionário que mapeia nomes das classes para IDs
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-large-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# Define os parâmetros do treinamento do modelo
training_args = TrainingArguments(
    output_dir="model_128tokens", # Diretório onde os modelos treinados serão salvos
    learning_rate=2e-5, # Taxa de aprendizado para otimização do modelo
    per_device_train_batch_size=8, # Tamanho do batch de treinamento por GPU/CPU
    per_device_eval_batch_size=8, # Tamanho do batch de avaliação por GPU/CPU
    num_train_epochs=10,  # Número de épocas de treinamento (quantas vezes o modelo verá todo o conjunto de dados)
    weight_decay=0.01, # Decaimento de peso para regularização e evitar overfitting
    eval_strategy="epoch", # Avaliação será feita ao final de cada época
    save_strategy="epoch", # Salvará o modelo ao final de cada época
    load_best_model_at_end=True, # Carrega o melhor modelo baseado na métrica de avaliação ao final do treinamento
    metric_for_best_model="accuracy", # Define que o melhor modelo será selecionado com base na maior acurácia obtida durante as avaliações.
    save_total_limit=2, # Mantém apenas os dois checkpoints mais recentes (ou melhores) no disco, economizando espaço de armazenamento.
    # push_to_hub=True,  # (Comentado) Caso ativado, envia o modelo para o Hugging Face Hub automaticamente
)

# Criação do Trainer, responsável pelo treinamento do modelo
trainer = Trainer(
    model=model,  # Modelo carregado anteriormente
    args=training_args,  # Argumentos de treinamento definidos acima
    train_dataset=tokenized_dataset["train"],  # Conjunto de treinamento tokenizado
    eval_dataset=tokenized_dataset["test"],  # Conjunto de teste tokenizado
    processing_class=tokenizer,  # Define o tokenizador para ser usado pelo Trainer 
    data_collator=data_collator,  # Gerencia o padding dos inputs para que fiquem no mesmo tamanho
    compute_metrics=compute_metrics,  # Função que calcula as métricas de avaliação do modelo
)

# Inicia o treinamento do modelo
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.119400,0.121628,0.976848
2,0.060900,0.130432,0.978342
3,0.032300,0.119317,0.978715
4,0.013300,0.098102,0.986557
5,0.010500,0.142417,0.982823
6,0.009700,0.129227,0.983943
7,0.005600,0.125223,0.985437
8,0.000000,0.133466,0.986557
9,0.000000,0.145568,0.986184
10,0.000500,0.150888,0.986184


TrainOutput(global_step=13390, training_loss=0.0289770239227555, metrics={'train_runtime': 374849.0548, 'train_samples_per_second': 0.286, 'train_steps_per_second': 0.036, 'total_flos': 9.982882493915136e+16, 'train_loss': 0.0289770239227555, 'epoch': 10.0})

In [16]:
# Obtém as previsões do modelo no conjunto de teste
predictions = trainer.predict(tokenized_dataset["test"])

# Extrai os logits (saída bruta do modelo antes da ativação softmax)
logits = predictions.predictions

# Converte os logits para rótulos preditos (classe com maior probabilidade)
preds = np.argmax(logits, axis=1)

# Converte os rótulos reais para numpy array
labels = np.array(tokenized_dataset["test"]['label'])

# Calcula a acurácia
accuracy = accuracy_score(labels, preds)

# Define os nomes das classes a partir do dicionário label2id
label_names = list(label2id.keys())

# Exibe os resultados
print(f'Accuracy: {accuracy:.4f}')
print(classification_report(labels, preds, target_names=label_names))

Accuracy: 0.9866
              precision    recall  f1-score   support

        FATO       0.98      0.98      0.98       889
        TESE       0.98      0.98      0.98       898
       RUIDO       1.00      1.00      1.00       891

    accuracy                           0.99      2678
   macro avg       0.99      0.99      0.99      2678
weighted avg       0.99      0.99      0.99      2678

